In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np

from urllib.request import urlopen
import json

import os.path


# Lotto 다음 회차 예측 프로그램
- 인공지능 분석 통한 로또 당첨 번호 예측
- 매주 월요일 진행하도록 크롬 작업 진행
    * 첫 시작 / 추가 진행 두 파트로 프로그램 진행되도록 코딩

### 1. API 통해 회차별 당첨번호
- returnValue : 요청결과
- totsellamnt : 총상금액
- drwNoDate : 날짜
- firstWinamnt : 1등 상금액
- firstPrzwnerCo : 1등 당첨자 수
- drwNo : 로또회차
- drwNo1 - 6 : 당첨번호 1 - 6
- bnusNo : 보너스 번호

In [2]:
#url = https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={회차}

response_test = urlopen("https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo=1")
response_test

In [3]:
decode_test = response_test.read().decode("utf-8")
data_json_test = json.loads(decode_test)
data_json_test

{'totSellamnt': 3681782000,
 'returnValue': 'success',
 'drwNoDate': '2002-12-07',
 'firstWinamnt': 0,
 'drwtNo6': 40,
 'drwtNo4': 33,
 'firstPrzwnerCo': 0,
 'drwtNo5': 37,
 'bnusNo': 16,
 'firstAccumamnt': 863604600,
 'drwNo': 1,
 'drwtNo2': 23,
 'drwtNo3': 29,
 'drwtNo1': 10}

In [4]:
data_json_test.values()

dict_values([3681782000, 'success', '2002-12-07', 0, 40, 33, 0, 37, 16, 863604600, 1, 23, 29, 10])

In [5]:
data_json_test.keys()

dict_keys(['totSellamnt', 'returnValue', 'drwNoDate', 'firstWinamnt', 'drwtNo6', 'drwtNo4', 'firstPrzwnerCo', 'drwtNo5', 'bnusNo', 'firstAccumamnt', 'drwNo', 'drwtNo2', 'drwtNo3', 'drwtNo1'])

In [16]:
data_json_test['returnValue']

'success'

In [6]:
df_test = json_normalize(data_json_test)
df_test

,totSellamnt,returnValue,drwNoDate,firstWinamnt,drwtNo6,drwtNo4,firstPrzwnerCo,drwtNo5,bnusNo,firstAccumamnt,drwNo,drwtNo2,drwtNo3,drwtNo1
0,3681782000,success,2002-12-07,0,40,33,0,37,16,863604600,1,23,29,10


In [7]:
# 2회차
response2 = urlopen("https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo=2")
decode2 = response2.read().decode("utf-8")
data_json2 = json.loads(decode2)
df_test_2 = json_normalize(data_json2)
df_test_2

,totSellamnt,returnValue,drwNoDate,firstWinamnt,drwtNo6,drwtNo4,firstPrzwnerCo,drwtNo5,bnusNo,firstAccumamnt,drwNo,drwtNo2,drwtNo3,drwtNo1
0,4904274000,success,2002-12-14,2002006800,42,25,1,32,2,0,2,13,21,9


In [8]:
dataframe = df_test.append(df_test_2)
dataframe

,totSellamnt,returnValue,drwNoDate,firstWinamnt,drwtNo6,drwtNo4,firstPrzwnerCo,drwtNo5,bnusNo,firstAccumamnt,drwNo,drwtNo2,drwtNo3,drwtNo1
0,3681782000,success,2002-12-07,0,40,33,0,37,16,863604600,1,23,29,10
0,4904274000,success,2002-12-14,2002006800,42,25,1,32,2,0,2,13,21,9


In [9]:
dataframe.drwNo.iloc[-1]

2

In [46]:
returnValue = "success"
cnt = 1
while returnValue == "success":
    print(cnt)
    print(returnValue)
    if cnt == 1:
        print("cnt 1 : start")
        response = urlopen(f"https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={cnt}")
        decode = response.read().decode("utf-8")
        data_json = json.loads(decode)
        datas = json_normalize(data_json)
        cnt += 1
        returnValue = data_json['returnValue']

    elif cnt != 1:
        print("cnt 2 - : start")
        response = urlopen(f"https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={cnt}")
        decode = response.read().decode("utf-8")
        data_json = json.loads(decode)
        data = json_normalize(data_json)
        datas = datas.append(data)
        cnt += 1
        returnValue = data_json['returnValue']
    

1
success
cnt 1 : start
2
success
cnt 2 - : start
3
success
cnt 2 - : start
4
success
cnt 2 - : start
5
success
cnt 2 - : start
6
success
cnt 2 - : start
7
success
cnt 2 - : start
8
success
cnt 2 - : start
9
success
cnt 2 - : start
10
success
cnt 2 - : start
11
success
cnt 2 - : start
12
success
cnt 2 - : start
13
success
cnt 2 - : start
14
success
cnt 2 - : start
15
success
cnt 2 - : start
16
success
cnt 2 - : start
17
success
cnt 2 - : start
18
success
cnt 2 - : start
19
success
cnt 2 - : start
20
success
cnt 2 - : start
21
success
cnt 2 - : start
22
success
cnt 2 - : start
23
success
cnt 2 - : start
24
success
cnt 2 - : start
25
success
cnt 2 - : start
26
success
cnt 2 - : start
27
success
cnt 2 - : start
28
success
cnt 2 - : start
29
success
cnt 2 - : start
30
success
cnt 2 - : start
31
success
cnt 2 - : start
32
success
cnt 2 - : start
33
success
cnt 2 - : start
34
success
cnt 2 - : start
35
success
cnt 2 - : start
36
success
cnt 2 - : start
37
success
cnt 2 - : start
38
success
c

In [47]:
datas

,totSellamnt,returnValue,drwNoDate,firstWinamnt,drwtNo6,drwtNo4,firstPrzwnerCo,drwtNo5,bnusNo,firstAccumamnt,drwNo,drwtNo2,drwtNo3,drwtNo1
0,3.681782e+09,success,2002-12-07,0.000000e+00,40.0,33.0,0.0,37.0,16.0,8.636046e+08,1.0,23.0,29.0,10.0
0,4.904274e+09,success,2002-12-14,2.002007e+09,42.0,25.0,1.0,32.0,2.0,0.000000e+00,2.0,13.0,21.0,9.0
0,4.729342e+09,success,2002-12-21,2.000000e+09,31.0,21.0,1.0,27.0,30.0,0.000000e+00,3.0,16.0,19.0,11.0
0,5.271464e+09,success,2002-12-28,0.000000e+00,42.0,31.0,0.0,40.0,2.0,1.267147e+09,4.0,27.0,30.0,14.0
0,6.277102e+09,success,2003-01-04,0.000000e+00,42.0,40.0,0.0,41.0,3.0,3.041095e+09,5.0,24.0,29.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1.080726e+11,success,2023-01-14,1.535083e+09,43.0,35.0,17.0,38.0,17.0,2.609642e+10,1050.0,12.0,31.0,6.0
0,1.192439e+11,success,2023-01-21,1.669558e+09,35.0,32.0,18.0,33.0,44.0,3.005205e+10,1051.0,26.0,30.0,21.0
0,1.075655e+11,success,2023-01-28,2.341683e+09,38.0,27.0,11.0,35.0,1.0,2.575851e+10,1052.0,17.0,26.0,5.0
0,1.137217e+11,success,2023-02-04,4.090367e+09,45.0,30.0,7.0,34.0,15.0,2.863257e+10,1053.0,26.0,29.0,22.0


In [48]:
# 1회차부터 수집해 returnValue가 False일 때까지 수집해 DF으로 만들고 CSV로 저장
def get_data():
    csv_path = "data/data.csv"
    cnt = 1
    while returnValue == "success":
        print(cnt)
        print(returnValue)
        if cnt == 1:
            print("cnt 1 : start")
            response = urlopen(f"https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={cnt}")
            decode = response.read().decode("utf-8")
            data_json = json.loads(decode)
            datas = json_normalize(data_json)
        elif cnt != 1:
            print("cnt 2 - : start")
            response = urlopen(f"https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={cnt}")
            decode = response.read().decode("utf-8")
            data_json = json.loads(decode)
            data = json_normalize(data_json)
            datas = datas.append(data)
        cnt += 1
        returnValue = data_json['returnValue']
    datas.to_csv(csv_path)

# 매주 월요일, 전 주 결과를 추가해 분석하고 당첨번호를 예측해야함
def add_data():
    # 지정된 경로에 csv파일이 있다면?
    csv_path = "data/data.csv"
    if os.path.isfile(csv_path) == True:
            df = pd.read_csv(csv_path)
            last_drwNo = df.drwNo.iloc[-1] 
            while data_json.returnValue == 'fail':
                response = urlopen(f"https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo={last_drwNo}")
                decode = response.read().decode("utf-8")
                data_json = json.loads(decode)
                data = json_normalize(data_json)
                df.append(data)
                last_drwNo += 1
    df.to_csv(csv_path)
    return df